# train simple

Now that you know how to pretrain a model, let’s go through the procedure to perform offline linear evaluation.

As for pretraining, we start by importing the required packages:

In [11]:
import torch
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from torchvision.models import resnet18

from solo.methods.linear import LinearModel  # imports the linear eval class
from solo.data.classification_dataloader import prepare_data

In [12]:
# basic parameters for offline linear evaluation
# some parameters for extra functionally are missing, but don't mind this for now.
kwargs = {
    "num_classes": 10,
    "cifar": True,
    "max_epochs": 100,
    "optimizer": "sgd",
    "precision": 16,
    "lars": False,
    "lr": 0.1,
    "exclude_bias_n_norm_lars": False,
    "gpus": "0",
    "weight_decay": 0,
    "extra_optimizer_args": {"momentum": 0.9},
    "scheduler": "step",
    "lr_decay_steps": [60, 80],
    "batch_size": 128,
    "num_workers": 4,
    "pretrained_feature_extractor": "path/to/pretrained/feature/extractor"
}

In [13]:
import torch.nn as nn

# create the backbone network
# the first convolutional and maxpooling layers of the ResNet backbone
# are adjusted to handle lower resolution images (32x32 instead of 224x224).
backbone = resnet18()
backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2, bias=False)
backbone.maxpool = nn.Identity()
backbone.fc = nn.Identity()

# load pretrained feature extractor
state = torch.load(kwargs["pretrained_feature_extractor"])["state_dict"]
for k in list(state.keys()):
    if "backbone" in k:
        state[k.replace("backbone.", "")] = state[k]
    del state[k]
backbone.load_state_dict(state, strict=False)

model = LinearModel(backbone, **kwargs)

/tmp/ipykernel_330/63356785.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(kwargs["pretrained_feature_extractor"])["state_dict"]


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/pretrained/feature/extractor'